In [1]:
import torch
import torch.nn as nn

In [2]:
%load_ext autoreload
%autoreload 2

In [23]:
umam_embeddings = torch.load("umam_embeddings.pt")
umdm_embeddings = torch.load("umdm_embeddings.pt")
umum_embeddings = torch.load("umum_embeddings.pt")
user_content_embedding = torch.load("user_content_based_embeddings.pt")

C:\Users\xiaoyicong\AppData\Local\Temp\ipykernel_27288\3309417524.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  umam_embeddings = torch.load("umam_embeddings.pt")
C:\U

In [48]:
len(umam_embeddings), len(umdm_embeddings), len(umum_embeddings), len(user_content_embedding)

(943, 943, 943, 943)

In [49]:
user_content_embedding[1]

tensor([[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0

In [50]:
umam_embeddings[1].size(), umdm_embeddings[1].size(), umum_embeddings[1].size(), user_content_embedding[1].size()

(torch.Size([1, 5089]),
 torch.Size([1, 5089]),
 torch.Size([1, 5089]),
 torch.Size([1, 825]))

In [51]:
torch.cat((umam_embeddings[1], umdm_embeddings[1], umum_embeddings[1]), dim = -1).shape

torch.Size([1, 15267])

## Prepare User and Movie Embedding

In [3]:
from utils.dataloader import DataLoader
data_loader = DataLoader(size="100k")

In [53]:
data_loader.load_items().head()

,movie_id,title,year,genre_unknown,genre_Action,genre_Adventure,genre_Animation,genre_Children's,genre_Comedy,genre_Crime,...,genre_Fantasy,genre_Film-Noir,genre_Horror,genre_Musical,genre_Mystery,genre_Romance,genre_Sci-Fi,genre_Thriller,genre_War,genre_Western
0,1,Toy Story,1995,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye,1995,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms,1995,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty,1995,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat,1995,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [8]:
from utils.data_split import train_test_split
import pandas as pd

In [9]:
movie_data = DataLoader(size="100k")

# Load rating data
data = movie_data.load_ratings()
data = data[['user', 'item', 'rating']]
train_list, test_list = train_test_split(data)
ratings = pd.concat([train_list, train_list], axis=0, ignore_index=True)

In [13]:
user_list = ratings['user'].unique().tolist()
item_list = ratings['item'].unique().tolist()

user2idx = {user: idx for idx, user in enumerate(user_list)}
idx2user = {idx: user for user, idx in user2idx.items()}


item2idx = {item: idx for idx, item in enumerate(item_list)}
idx2item = {idx: item for item, idx in item2idx.items()}

user2idx

{97: 0,
 266: 1,
 811: 2,
 24: 3,
 31: 4,
 281: 5,
 569: 6,
 260: 7,
 332: 8,
 324: 9,
 423: 10,
 468: 11,
 287: 12,
 894: 13,
 869: 14,
 639: 15,
 539: 16,
 500: 17,
 482: 18,
 335: 19,
 849: 20,
 771: 21,
 926: 22,
 40: 23,
 364: 24,
 765: 25,
 308: 26,
 445: 27,
 714: 28,
 705: 29,
 77: 30,
 818: 31,
 596: 32,
 372: 33,
 166: 34,
 756: 35,
 251: 36,
 883: 37,
 437: 38,
 240: 39,
 108: 40,
 68: 41,
 175: 42,
 159: 43,
 140: 44,
 755: 45,
 732: 46,
 307: 47,
 533: 48,
 695: 49,
 803: 50,
 64: 51,
 236: 52,
 766: 53,
 900: 54,
 87: 55,
 693: 56,
 724: 57,
 779: 58,
 517: 59,
 661: 60,
 588: 61,
 327: 62,
 219: 63,
 210: 64,
 657: 65,
 71: 66,
 929: 67,
 650: 68,
 469: 69,
 847: 70,
 200: 71,
 480: 72,
 448: 73,
 522: 74,
 676: 75,
 941: 76,
 137: 77,
 209: 78,
 626: 79,
 214: 80,
 573: 81,
 356: 82,
 622: 83,
 73: 84,
 478: 85,
 45: 86,
 892: 87,
 591: 88,
 614: 89,
 507: 90,
 267: 91,
 169: 92,
 333: 93,
 452: 94,
 310: 95,
 261: 96,
 607: 97,
 939: 98,
 823: 99,
 434: 100,
 328: 101,

In [14]:
item2idx

{228: 0,
 222: 1,
 670: 2,
 89: 3,
 484: 4,
 98: 5,
 50: 6,
 7: 7,
 435: 8,
 655: 9,
 168: 10,
 466: 11,
 432: 12,
 429: 13,
 79: 14,
 430: 15,
 174: 16,
 192: 17,
 32: 18,
 357: 19,
 202: 20,
 189: 21,
 1: 22,
 919: 23,
 135: 24,
 204: 25,
 431: 26,
 526: 27,
 197: 28,
 96: 29,
 205: 30,
 191: 31,
 408: 32,
 663: 33,
 82: 34,
 434: 35,
 603: 36,
 195: 37,
 23: 38,
 100: 39,
 133: 40,
 83: 41,
 169: 42,
 186: 43,
 69: 44,
 173: 45,
 1126: 46,
 423: 47,
 874: 48,
 508: 49,
 272: 50,
 124: 51,
 313: 52,
 283: 53,
 268: 54,
 286: 55,
 275: 56,
 14: 57,
 321: 58,
 319: 59,
 237: 60,
 325: 61,
 276: 62,
 924: 63,
 9: 64,
 304: 65,
 294: 66,
 748: 67,
 690: 68,
 258: 69,
 323: 70,
 289: 71,
 307: 72,
 315: 73,
 300: 74,
 308: 75,
 988: 76,
 678: 77,
 243: 78,
 901: 79,
 763: 80,
 367: 81,
 742: 82,
 372: 83,
 662: 84,
 25: 85,
 178: 86,
 318: 87,
 56: 88,
 216: 89,
 475: 90,
 109: 91,
 117: 92,
 127: 93,
 55: 94,
 238: 95,
 180: 96,
 486: 97,
 176: 98,
 729: 99,
 421: 100,
 8: 101,
 582: 102

In [17]:
user_ids = user_list
item_ids = item_list

len(user_ids), len(item_ids)

(943, 1682)

In [18]:
items_df = data_loader.load_items()

In [19]:
'''Generate one-hot encoding for movie genre'''

import pandas as pd


# Number of genre columns (k)
num_genres = len(items_df.columns) - 1

# Create a dictionary to store the hot encodings
hot_encoding_dict = {}

# Iterate over each row in the dataframe
for index, row in items_df.iterrows():
    movie_id = row['movie_id']
    genres = [row[col] for col in row.index if col.startswith('genre_')]

    # Create hot encoding for the genres
    hot_encoding = [1 if genre == 1 else 0 for genre in genres]

    # Store the hot encoding in the dictionary
    hot_encoding_dict[movie_id] = torch.tensor(hot_encoding).unsqueeze(0)

print(hot_encoding_dict)

torch.save(hot_encoding_dict, "movie_genre_hot_embeddings.pt")

{1: tensor([[0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 2: tensor([[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]]), 3: tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]]), 4: tensor([[0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 5: tensor([[0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]]), 6: tensor([[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 7: tensor([[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]]), 8: tensor([[0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 9: tensor([[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 10: tensor([[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]]), 11: tensor([[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]]), 12: tensor([[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]]), 13: tensor([[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 14: tensor([[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0

In [20]:
user_ids

[97,
 266,
 811,
 24,
 31,
 281,
 569,
 260,
 332,
 324,
 423,
 468,
 287,
 894,
 869,
 639,
 539,
 500,
 482,
 335,
 849,
 771,
 926,
 40,
 364,
 765,
 308,
 445,
 714,
 705,
 77,
 818,
 596,
 372,
 166,
 756,
 251,
 883,
 437,
 240,
 108,
 68,
 175,
 159,
 140,
 755,
 732,
 307,
 533,
 695,
 803,
 64,
 236,
 766,
 900,
 87,
 693,
 724,
 779,
 517,
 661,
 588,
 327,
 219,
 210,
 657,
 71,
 929,
 650,
 469,
 847,
 200,
 480,
 448,
 522,
 676,
 941,
 137,
 209,
 626,
 214,
 573,
 356,
 622,
 73,
 478,
 45,
 892,
 591,
 614,
 507,
 267,
 169,
 333,
 452,
 310,
 261,
 607,
 939,
 823,
 434,
 328,
 731,
 886,
 303,
 618,
 61,
 248,
 708,
 410,
 211,
 670,
 389,
 67,
 352,
 276,
 524,
 560,
 381,
 262,
 449,
 897,
 586,
 255,
 282,
 853,
 498,
 121,
 111,
 616,
 425,
 885,
 193,
 893,
 930,
 542,
 371,
 589,
 426,
 138,
 312,
 729,
 306,
 706,
 508,
 523,
 863,
 674,
 395,
 535,
 79,
 599,
 794,
 781,
 430,
 606,
 529,
 854,
 66,
 142,
 785,
 527,
 775,
 627,
 311,
 745,
 681,
 383,
 612,
 

In [ ]:
user_input_emb = {}
item_input_emb = {}

for user_id in user_ids:
    user_input_emb[user_id] = torch.cat((umam_embeddings[user_id], umdm_embeddings[user_id], umum_embeddings[user_id]), dim = 1)


item_input_emb_unordered = torch.load("movie_genre_hot_embeddings.pt")

for item_id in item_ids:
    item_input_emb[item_id] = item_input_emb_unordered[item_id]

C:\Users\xiaoyicong\AppData\Local\Temp\ipykernel_27288\721580580.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  item_input_emb = torch.load("movie_genre_hot_embeddings.

In [ ]:
item

1682

In [27]:
user_input_emb

{97: tensor([[  0., 193., 115.,  ...,  44., 125.,  58.]]),
 266: tensor([[ 0., 58., 26.,  ..., 21., 45., 13.]]),
 811: tensor([[ 0., 72., 32.,  ..., 19., 42.,  7.]]),
 24: tensor([[  0., 271., 114.,  ...,  50., 136.,  62.]]),
 31: tensor([[ 0., 76., 36.,  ..., 18., 70., 16.]]),
 281: tensor([[ 0., 88., 40.,  ..., 26., 50.,  7.]]),
 569: tensor([[  0., 312., 130.,  ...,  56., 140.,  47.]]),
 260: tensor([[ 0., 93., 27.,  ..., 20., 40.,  8.]]),
 332: tensor([[   0., 1102.,  466.,  ...,  112.,  363.,  127.]]),
 324: tensor([[  0., 297.,  92.,  ...,  59., 132.,  27.]]),
 423: tensor([[  0., 308., 105.,  ...,  55., 124.,  22.]]),
 468: tensor([[  0., 495., 215.,  ...,  92., 284., 120.]]),
 287: tensor([[  0., 319., 122.,  ...,  49., 124.,  50.]]),
 894: tensor([[  0., 729., 270.,  ..., 167., 445., 112.]]),
 869: tensor([[  0., 171.,  69.,  ...,  33.,  93.,  23.]]),
 639: tensor([[  0., 310., 169.,  ...,  77., 289., 119.]]),
 539: tensor([[  0., 118.,  50.,  ...,  35., 109.,  37.]]),
 500: t

In [67]:
user_input_emb = torch.cat(tuple([user_input_emb[i] for i in user_input_emb]), dim = 0).float()
item_input_emb = torch.cat(tuple([item_input_emb[i] for i in item_input_emb]), dim = 0).float()

user_input_emb.shape, item_input_emb.shape

(torch.Size([943, 15267]), torch.Size([1682, 19]))

## Backbone

In [69]:
embedding_dim = 64


# Pretrained embeddings
user_pretrained_embedding = nn.Embedding(num_embeddings = len(user_ids), embedding_dim = embedding_dim)
item_pretrained_embedding = nn.Embedding(num_embeddings = len(item_ids), embedding_dim = embedding_dim)

# Project user_input_emb and item_input_emb to embedding_dim
user_input_MLP = nn.Linear(user_input_emb.shape[1], embedding_dim)
item_input_MLP = nn.Linear(item_input_emb.shape[1], embedding_dim)

# Concat pretrained and input embedding
user_input_emb_2 = user_input_MLP(user_input_emb)
item_input_emb_2 = item_input_MLP(item_input_emb)

user_embedding = torch.cat((user_pretrained_embedding.weight, user_input_emb_2), dim = -1)
item_embedding = torch.cat((item_pretrained_embedding.weight, item_input_emb_2), dim = -1)

In [73]:
user_embedding.size(), item_embedding.size()

(torch.Size([943, 128]), torch.Size([1682, 128]))

In [88]:
torch.save(user_pretrained_embedding.weight, "user_pretrained_embedding.pt")
torch.save(item_pretrained_embedding.weight, "item_pretrained_embedding.pt")

In [91]:
user_embedding, nn.Embedding.from_pretrained(user_embedding)(torch.tensor([0]))

(tensor([[ 1.1741e+00,  1.7420e-01,  8.8006e-01,  ..., -7.2083e+01,
           8.5437e+01, -5.7284e+00],
         [-1.4877e+00,  7.9509e-01, -3.4168e-01,  ..., -1.6482e+02,
           1.9473e+02, -1.0673e+01],
         [-2.5587e+00,  1.1711e+00, -9.0951e-01,  ..., -2.4612e+02,
           2.7303e+02, -1.6895e+01],
         ...,
         [-4.1602e-01,  4.9267e-01,  7.6463e-01,  ..., -1.1972e+02,
           1.3837e+02, -6.9562e+00],
         [-1.2013e+00, -2.3896e-03, -8.9671e-02,  ..., -4.4966e+01,
           5.9529e+01, -1.2013e+00],
         [ 4.2495e-01, -1.9999e+00, -9.2644e-01,  ..., -4.1952e+01,
           4.9369e+01,  5.4042e-02]], grad_fn=<CatBackward0>),
 tensor([[ 1.1741e+00,  1.7420e-01,  8.8006e-01,  5.0402e-01, -1.1589e+00,
           5.6583e-01, -2.8851e-02, -1.2438e-01,  1.8789e+00,  7.9068e-01,
          -4.7534e-01,  5.9629e-02,  8.1550e-01,  5.3771e-01, -5.3345e-01,
          -1.5870e+00,  9.9692e-01, -2.8098e-01, -9.4895e-01, -2.9070e+00,
          -1.9732e+00, -2.8063

In [12]:
from models.recommender import RecommenderModel
import os

rec_model = RecommenderModel(
        size="100k",
        num_layers=3,
        num_negatives=4,
        embedding_dim=64,
        learning_rate=0.01,
        epochs=500,
        batch_size=1024,
        device='cpu',
        input_embedding_paths={"umam": os.path.join("data", "umam_embeddings.pt"),
                               "umdm": os.path.join("data", "umdm_embeddings.pt"),
                               "umum": os.path.join("data", "umum_embeddings.pt"),
                               "user_content": os.path.join("data", "user_content_based_embeddings.pt"),
                               "item_content": os.path.join("data", "movie_genre_hot_embeddings.pt"),
                               "user_pretrained": os.path.join("data", "user_pretrained_embedding.pt"),
                               "item_pretrained": os.path.join("data", "item_pretrained_embedding.pt") }
    )

rec_model.prepare_training_data()

2024-11-20 20:28:53,912 - Recommender model - [INFO]: Using CPU
2024-11-20 20:28:53,914 - Recommender model - [INFO]: Preparing data...
2024-11-20 20:28:54,163 - Recommender model - [INFO]: Data Loaded.
2024-11-20 20:28:54,815 - Recommender model - [INFO]: Generating negative samples...
2024-11-20 20:28:55,511 - Recommender model - [INFO]: Generating negative samples...
d:\NUS\CS5284_graph_ml\project\dinghui101\models\recommender.py:182: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded vi

In [99]:
rec_model.model.parameters

<bound method Module.parameters of Recommender(
  (mlp_u): Linear(in_features=16092, out_features=64, bias=True)
  (mlp_i): Linear(in_features=19, out_features=64, bias=True)
  (transformer_layer): TransformerLayer(
    (attention): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
    )
  )
)>

In [13]:
from utils.metrics import calculate_ranking_metrics, calculate_rating_metrics

rec_model.prepare_training_data()
rec_model.train()
predictions = rec_model.predict()

ratings = calculate_rating_metrics(rec_model.test_pre, predictions)

top_k_scores = rec_model.recommend_k()
rankings = calculate_ranking_metrics(rec_model.test_pre, top_k_scores, 10)

2024-11-20 20:30:50,807 - Recommender model - [INFO]: Preparing data...
2024-11-20 20:30:51,240 - Recommender model - [INFO]: Data Loaded.
2024-11-20 20:30:51,663 - Recommender model - [INFO]: Generating negative samples...
2024-11-20 20:30:52,301 - Recommender model - [INFO]: Generating negative samples...
d:\NUS\CS5284_graph_ml\project\dinghui101\models\recommender.py:182: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user v

Epoch 1/500, BPR Loss: 0.7095


2024-11-20 20:30:53,759 - Recommender model - [INFO]: Epoch 2/500, BPR Loss: 0.6827


Epoch 2/500, BPR Loss: 0.6827


2024-11-20 20:30:54,232 - Recommender model - [INFO]: Epoch 3/500, BPR Loss: 0.6749


...

2024-11-20 20:31:06,293 - Recommender model - [INFO]: Epoch 29/500, BPR Loss: 0.6677


Epoch 29/500, BPR Loss: 0.6677


2024-11-20 20:31:06,648 - Recommender model - [INFO]: Epoch 30/500, BPR Loss: 0.6680


Epoch 30/500, Recall@K: 0.02014846235418876


KeyboardInterrupt: 

In [5]:
lgcn_model.model.parameters

<bound method Module.parameters of LightGCN(
  (user_embedding): Embedding(943, 64)
  (item_embedding): Embedding(1682, 64)
)>

In [4]:
from models.lightgcn_model_exp import LightGCNModel2
import torch
import torch.nn as nn

lgcn_model2 = LightGCNModel2(
        size="100k",
        num_layers=3,
        num_negatives=4,
        embedding_dim=64,
        learning_rate=0.01,
        epochs=1000,
        batch_size=1024,
        device='cpu'
    )

lgcn_model2.prepare_training_data()

2024-11-20 17:43:44,489 - LGCN2 model - [INFO]: Using CPU
2024-11-20 17:43:44,489 - LGCN2 model - [INFO]: Preparing data...
2024-11-20 17:43:44,740 - LGCN2 model - [INFO]: Data Loaded.
2024-11-20 17:43:45,222 - LGCN2 model - [INFO]: Generating negative samples...
2024-11-20 17:43:45,740 - LGCN2 model - [INFO]: Generating negative samples...
d:\NUS\CS5284_graph_ml\project\dinghui101\models\lightgcn_model_exp.py:173: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they

In [5]:
lgcn_model2.model.parameters

<bound method Module.parameters of LightGCN2(
  (mlp_u): Linear(in_features=16092, out_features=64, bias=True)
  (mlp_i): Linear(in_features=19, out_features=64, bias=True)
)>

In [6]:
from utils.metrics import calculate_ranking_metrics, calculate_rating_metrics

lgcn_model2.train()
predictions = lgcn_model2.predict()

ratings = calculate_rating_metrics(lgcn_model2.test_pre, predictions)

top_k_scores = lgcn_model2.recommend_k()
rankings = calculate_ranking_metrics(lgcn_model2.test_pre, top_k_scores, 10)

2024-11-20 17:43:57,658 - LGCN2 model - [INFO]: Epoch 1/1000, BPR Loss: 5.1499
2024-11-20 17:43:57,959 - LGCN2 model - [INFO]: Epoch 2/1000, BPR Loss: 5.1676
2024-11-20 17:43:58,271 - LGCN2 model - [INFO]: Epoch 3/1000, BPR Loss: 5.1679
2024-11-20 17:43:58,637 - LGCN2 model - [INFO]: Epoch 4/1000, BPR Loss: 5.1680
2024-11-20 17:43:58,958 - LGCN2 model - [INFO]: Epoch 5/1000, BPR Loss: 5.1681
2024-11-20 17:43:59,287 - LGCN2 model - [INFO]: Epoch 6/1000, BPR Loss: 5.1680
2024-11-20 17:43:59,587 - LGCN2 model - [INFO]: Epoch 7/1000, BPR Loss: 5.1681
2024-11-20 17:43:59,915 - LGCN2 model - [INFO]: Epoch 8/1000, BPR Loss: 5.1683
2024-11-20 17:44:00,197 - LGCN2 model - [INFO]: Epoch 9/1000, BPR Loss: 5.1682
2024-11-20 17:44:00,490 - LGCN2 model - [INFO]: Epoch 10/1000, BPR Loss: 5.1683
2024-11-20 17:44:00,777 - LGCN2 model - [INFO]: Epoch 11/1000, BPR Loss: 5.1683
2024-11-20 17:44:01,054 - LGCN2 model - [INFO]: Epoch 12/1000, BPR Loss: 5.1683
2024-11-20 17:44:01,354 - LGCN2 model - [INFO]: E

In [7]:
rankings

{'MAP': 0.05243953065703742,
 'nDCG@k': 0.22266731254299707,
 'Precision@k': 0.1897136797454931,
 'Recall@k': 0.0939751882608445}

In [1]:
from models.lightgcn_model import LightGCNModel

lgcn_model = LightGCNModel(
        size="100k",
        num_layers=3,
        num_negatives=4,
        embedding_dim=64,
        learning_rate=0.01,
        epochs=1000,
        batch_size=1024,
        device='cpu'
    )

lgcn_model.prepare_training_data()

lgcn_model.train()


2024-11-20 16:53:02,339 - LGCN model - [INFO]: Using CPU
2024-11-20 16:53:02,339 - LGCN model - [INFO]: Preparing data...
2024-11-20 16:53:02,592 - LGCN model - [INFO]: Data Loaded.
2024-11-20 16:53:03,106 - LGCN model - [INFO]: Generating negative samples...
2024-11-20 16:53:03,609 - LGCN model - [INFO]: Generating negative samples...
2024-11-20 16:53:03,822 - LGCN model - [INFO]: building graph...
2024-11-20 16:53:03,839 - LGCN model - [INFO]: initilizing model...
2024-11-20 16:53:04,156 - LGCN model - [INFO]: Epoch 1/1000, BPR Loss: 0.6931
2024-11-20 16:53:04,376 - LGCN model - [INFO]: Epoch 2/1000, BPR Loss: 0.6924
2024-11-20 16:53:04,608 - LGCN model - [INFO]: Epoch 3/1000, BPR Loss: 0.6900
2024-11-20 16:53:04,823 - LGCN model - [INFO]: Epoch 4/1000, BPR Loss: 0.6852
2024-11-20 16:53:05,091 - LGCN model - [INFO]: Epoch 5/1000, BPR Loss: 0.6775
2024-11-20 16:53:05,306 - LGCN model - [INFO]: Epoch 6/1000, BPR Loss: 0.6668
2024-11-20 16:53:05,542 - LGCN model - [INFO]: Epoch 7/1000, 

In [2]:
from utils.metrics import calculate_ranking_metrics, calculate_rating_metrics

predictions = lgcn_model.predict()

ratings = calculate_rating_metrics(lgcn_model.test_pre, predictions)

top_k_scores = lgcn_model.recommend_k()
rankings = calculate_ranking_metrics(lgcn_model.test_pre, top_k_scores, 10)

rankings

{'MAP': 0.11230943829402501,
 'nDCG@k': 0.3609482703371832,
 'Precision@k': 0.3031813361611877,
 'Recall@k': 0.19674492206266433}